In [ ]:
#!pip install hanlp
#!pip install simalign

In [7]:
import os, csv

import pandas as pd
from tqdm import tqdm
from simalign import SentenceAligner
# from google.cloud import translate_v2 as translate

/usr/local/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [22]:

src_lan = 'en'
# target language
tgt_lan = 'BN'
language = "Bengali"

RESOURCE = '~/Desktop/Research/Sense-Projection'
DATA =  RESOURCE+'/data/'+language+"/second"
TRANS_FILE = os.path.join(DATA, 'gold-sentences-'+language+'.tsv')
TOKENS_FILE = os.path.join(DATA, language+'-Gold2.tsv')
ALIGN_SENT_FILE = os.path.join(DATA, 'sentences-aligned-'+language+'.tsv')
ALIGN_TOKEN_FILE = os.path.join(DATA, 'tokens-aligned-'+language+'.tsv')

# DATA = os.path.join(RESOURCE, 'data')
DATA =  RESOURCE+'/data/English/Original'

KEY = os.path.join(RESOURCE, 'key')
RAW_SENSE_TSV = os.path.join(DATA, 'full-tokens-English-wSenses.tsv')
RAW_TOKENS_TSV =  os.path.join(DATA, 'full-tokens-English.tsv')
RAW_SENTENCE_TSV = os.path.join(DATA, 'gold-sentences-English-full.tsv')


# DATA

In [23]:
# raw
raw_sense_df = pd.read_csv(RAW_SENSE_TSV, delimiter='\t', quoting=csv.QUOTE_NONE)
raw_sense_df.head()

,Token ID,Token,Lemma,POS,BN Synset
0,d001.s001.t001,This,NaN,X,NaN
1,d001.s001.t002,document,document,N,bn:00028015n;wiki:document;wn:document%1:10:00...
2,d001.s001.t003,is,be,V,NaN
3,d001.s001.t004,a,NaN,X,NaN
4,d001.s001.t005,summary,summary,N,bn:00075142n;wiki:summary_(law);wn:sum-up%1:10...


In [24]:
sentence_df = pd.read_csv(RAW_SENTENCE_TSV, delimiter='\t',  quoting=csv.QUOTE_NONE)
sentence_df.head()
sents = sentence_df["ID"].tolist()
len(sents)
# print(sents)

138

In [25]:
sids = sentence_df["ID"].tolist()
# sids = [i.replace('d', 'i') for i in sids]
# sids[:4]
print(len(sids))

138


In [26]:
trans_df = pd.read_csv(TRANS_FILE, delimiter='\t', quoting=csv.QUOTE_NONE)
trans_df.head()

,Sentence ID,Sentence
0,d001.s001,এইনথিটিইউরোপীয়পাবলিকঅ্যাসেসমেন্টরিপোর্টের(ইপি...
1,d001.s002,কমিটিফরমেডিসিনালপ্রোডাক্টসফরহিউম্যানইউজ(সিএইচএ...
2,d001.s003,আপনারচিকিৎসাসংক্রান্তঅবস্থাঅথবাআপনারচিকিৎসারবি...
3,d001.s004,"যদিআপনিসিএইচএমপিরসুপারিশেরভিত্তিতেআরোতথ্যচান,ত..."
4,d001.s005,আলিমতাকি?


In [27]:
tkns_df = pd.read_csv(TOKENS_FILE, delimiter='\t', quoting=csv.QUOTE_NONE)
tkns_df.head(1000)

,Token ID,Token,Lemma,POS,BN Synset,Gloss
0,d001.s001.t001,এই,এই,X,NaN,NaN
1,d001.s001.t002,নথিটি,নথি,N,bn:00028015n,Writing that provides information (especially ...
2,d001.s001.t003,ইউরোপীয়,ইউরোপ,N,NaN,NaN
3,d001.s001.t004,পাবলিক,পাবলিক,A,bn:00109211a,Not private; open to or concerning the people ...
4,d001.s001.t005,অ্যাসেসমেন্ট,অ্যাসেসমেন্ট,N,bn:00006502n,The act of judging or assessing a person or si...
...,...,...,...,...,...,...
995,d002.s020.t022,গণ্য,গণ্য,V,bn:00095141v,Regard or consider in a specific way
996,d002.s020.t023,হবে,হবে,V,NaN,NaN
997,d002.s020.t024,।,।,X,NaN,NaN
998,d002.s021.t001,:,:,X,NaN,NaN


In [28]:
tgt_tkns = tkns_df['Token'].tolist()
len(tgt_tkns)

2283

# ALIGNMENT

In [29]:
device = 'cpu'
layer = 8
# "bert": "bert-base-multilingual-cased",
# "xlmr": "xlm-roberta-base"
model = 'xlmr'
# mwmf (Match), inter (ArgMax), itermax (IterMax)
mode = 'inter'
matching_methods = 'mai'
token_type='bpe'
aligner = SentenceAligner(
    model=model
    , token_type=token_type
    , matching_methods=matching_methods
    , layer=layer
    , device=device
)

/usr/local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-09-14 02:05:57,629 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: xlm-roberta-base


In [30]:
raw_sense_df.head()

,Token ID,Token,Lemma,POS,BN Synset
0,d001.s001.t001,This,NaN,X,NaN
1,d001.s001.t002,document,document,N,bn:00028015n;wiki:document;wn:document%1:10:00...
2,d001.s001.t003,is,be,V,NaN
3,d001.s001.t004,a,NaN,X,NaN
4,d001.s001.t005,summary,summary,N,bn:00075142n;wiki:summary_(law);wn:sum-up%1:10...


In [31]:
# english token sequences
idxes = raw_sense_df['Token ID'].tolist()
tokens = raw_sense_df['Token'].tolist()
# print(idxes,tokens)
en_sents = []
pre_sid = None
s = []
for i, t in zip(idxes, tokens):
    # print(i)
    sid = i.split('.')[1]
    if sid != pre_sid:
        if s:
            en_sents.append(s)
        pre_sid = sid
        s = [t]
    else:
        s.append(t)
en_sents.append(s)

print(len(en_sents))

# en_sents2 = trans_df["English"].tolist()
# pre_sid = None
# sent = []
# for i, s in zip(idxes, tokens):
#     if s != "-end-":
#         sent.append(s)
#     else:
#         en_sents.append(sent)
#         sent = []
# print(len(en_sents))
# for x in range(len(en_sents)):
#     print(" ".join(en_sents[x]),en_sents2[x])
#     if " ".join(en_sents[x]) != en_sents2[x]:
#         print(x)

138


In [32]:
print(en_sents[0])

['This', 'document', 'is', 'a', 'summary', 'of', 'the', 'European', 'Public', 'Assessment', 'Report', '(', 'EPAR', ')', '.']


In [33]:
# Italian token sequences
import re
idxes = tkns_df['Token ID'].tolist()
tokens = tkns_df['Token'].tolist()

tgt_sents = []
# en_sents2 = trans_df["English"].tolist()
# pre_sid = None
# sent = []
# for i, s in zip(idxes, tokens):
#     if s != "-end-":
#         sent.append(s)
#     else:
#         tgt_sents.append(sent)
#         sent = []
# print(tgt_sents)
pre_sid = None
s = []
for i, t in zip(idxes, tokens):
    # print(t)
    # print(i)
    sid = i.split('.')[1]
    # if "\u200c" in t:
    #     t= re.sub(r'\u200c', ' ', t)
    if sid != pre_sid:
        if s:
            tgt_sents.append(s)
        pre_sid = sid
        s = [t]
    else:
        s.append(t)
tgt_sents.append(s)
# print(tgt_sents)
print(len(tgt_sents))

138


In [34]:
print(tgt_sents[92])

['ফাউন্ডেশন', 'সম্প্রতি', 'একটি', 'সম্মেলন', 'আয়োজনের', 'সাথে', 'জড়িত', 'যা', 'এই', 'কঠিন', 'চ্যালেঞ্জ', 'মোকাবেলা', 'করেছে', '।']


In [35]:
# do the alignment
aligns = []
for en_s, tgt_s in zip(tqdm(en_sents), tgt_sents):
    # print(en_s,tgt_s,'\n')
    alignments = aligner.get_word_aligns(en_s, tgt_s)
    align = ''
    for w1, w2 in alignments[mode]:
        align += f'{w1}-{w2} '
    align = align.strip()
    aligns.append(align)

100%|██████████| 138/138 [00:51<00:00,  2.68it/s]


In [37]:
aligns[0]

'0-0 4-9 7-2 8-3 10-5 11-6 12-7 13-8 14-9 14-10'

In [38]:
print(len(aligns))
trans_df['English-'+language+' Alignment'] = aligns
trans_df.head(200)

138


,Sentence ID,Sentence,English-Bengali Alignment
0,d001.s001,এইনথিটিইউরোপীয়পাবলিকঅ্যাসেসমেন্টরিপোর্টের(ইপি...,0-0 4-9 7-2 8-3 10-5 11-6 12-7 13-8 14-9 14-10
1,d001.s002,কমিটিফরমেডিসিনালপ্রোডাক্টসফরহিউম্যানইউজ(সিএইচএ...,4-0 6-2 11-7 12-8 13-9 14-11 16-10 21-20 22-21...
2,d001.s003,আপনারচিকিৎসাসংক্রান্তঅবস্থাঅথবাআপনারচিকিৎসারবি...,0-8 2-11 3-9 4-10 5-7 6-0 8-3 9-4 11-6 12-13 1...
3,d001.s004,"যদিআপনিসিএইচএমপিরসুপারিশেরভিত্তিতেআরোতথ্যচান,ত...",0-0 1-1 2-7 3-5 4-6 6-4 7-4 12-8 13-16 15-10 1...
4,d001.s005,আলিমতাকি?,2-0 3-2
...,...,...,...
133,d004.s020,"এছাড়া,বমিকরারচিকিৎসাশুধুঅন্যান্যসহায়কব্যবস্থ...",0-1 1-2 3-5 5-3 6-15 7-6 8-14 10-13 11-7 12-8 ...
134,d004.s021,সেরেনিয়ারসাথেকোনঝুঁকিজড়িত?,0-2 3-3 4-4 5-1 6-0 7-5
135,d004.s022,দৈনিকডোজেঅতিরিক্তসময়েরজন্যসুপারিশকৃতডোজেরচেয়...,0-1 0-22 1-27 2-23 3-24 4-25 6-5 7-0 12-8 13-9...
136,d004.s023,"যে-ব্যক্তিওষুধদেনঅথবাপশুরসংস্পর্শেআসেন,তারজন্য...",0-10 3-11 4-9 8-2 10-1 11-3 12-6 17-4 18-15


In [39]:
trans_df.to_csv(ALIGN_SENT_FILE, sep='\t', index=False)

# ALIGN TOKENS IN TOKEN FILE

In [40]:
tkns_df.head()


,Token ID,Token,Lemma,POS,BN Synset,Gloss
0,d001.s001.t001,এই,এই,X,NaN,NaN
1,d001.s001.t002,নথিটি,নথি,N,bn:00028015n,Writing that provides information (especially ...
2,d001.s001.t003,ইউরোপীয়,ইউরোপ,N,NaN,NaN
3,d001.s001.t004,পাবলিক,পাবলিক,A,bn:00109211a,Not private; open to or concerning the people ...
4,d001.s001.t005,অ্যাসেসমেন্ট,অ্যাসেসমেন্ট,N,bn:00006502n,The act of judging or assessing a person or si...


In [41]:
aligned_tks_df = pd.read_csv(RAW_SENSE_TSV, delimiter='\t', quoting=csv.QUOTE_NONE)
aligned_tks_df.head()

,Token ID,Token,Lemma,POS,BN Synset
0,d001.s001.t001,This,NaN,X,NaN
1,d001.s001.t002,document,document,N,bn:00028015n;wiki:document;wn:document%1:10:00...
2,d001.s001.t003,is,be,V,NaN
3,d001.s001.t004,a,NaN,X,NaN
4,d001.s001.t005,summary,summary,N,bn:00075142n;wiki:summary_(law);wn:sum-up%1:10...


In [42]:
aligned_tks_df['Token ID '+tgt_lan] = None
aligned_tks_df['Token '+tgt_lan] = None
aligned_tks_df['Lemma '+tgt_lan] = None
aligned_tks_df['POS '+tgt_lan] = None
aligned_tks_df['Sense '+tgt_lan] = None

In [43]:
aligned_tks_df.head()
print(len(aligned_tks_df['Token']))

2638


In [44]:
trans_df.head()

,Sentence ID,Sentence,English-Bengali Alignment
0,d001.s001,এইনথিটিইউরোপীয়পাবলিকঅ্যাসেসমেন্টরিপোর্টের(ইপি...,0-0 4-9 7-2 8-3 10-5 11-6 12-7 13-8 14-9 14-10
1,d001.s002,কমিটিফরমেডিসিনালপ্রোডাক্টসফরহিউম্যানইউজ(সিএইচএ...,4-0 6-2 11-7 12-8 13-9 14-11 16-10 21-20 22-21...
2,d001.s003,আপনারচিকিৎসাসংক্রান্তঅবস্থাঅথবাআপনারচিকিৎসারবি...,0-8 2-11 3-9 4-10 5-7 6-0 8-3 9-4 11-6 12-13 1...
3,d001.s004,"যদিআপনিসিএইচএমপিরসুপারিশেরভিত্তিতেআরোতথ্যচান,ত...",0-0 1-1 2-7 3-5 4-6 6-4 7-4 12-8 13-16 15-10 1...
4,d001.s005,আলিমতাকি?,2-0 3-2


In [45]:
def process_nan(s):
  if type(s)!=type(''):
    return ''
  return s

In [46]:
count_en = 0
sentence_num = 0
count_es = 0
for i, row in trans_df.iterrows():
    # print(row.iloc[0])
    sent_i = row.iloc[0]
    # print(sent_i)
    # print(sent_i)
    tgt_sent_i = sent_i
    # print(row)
    # sent_i = sent_i.replace("i","d")
    # tgt_sent = tkns_df.loc[count_es:count_es+len(tgt_sents[sentence_num])]
    # en_sent = aligned_tks_df.loc[count_en:count_en+len(en_sents[sentence_num])]
    # print(tgt_sent.loc[2])
    # print(row)
    for alignment in row['English-'+language+' Alignment'].split():
        try:
            w1, w2 = alignment.split('-')
            
            en_id = f'{sent_i}.t{int(w1) + 1:03d}'
            tgt_id = f'{tgt_sent_i}.t{int(w2) + 1:03d}'
            row_en = aligned_tks_df[aligned_tks_df['Token ID'] == en_id]
            row_tgt = tkns_df[tkns_df['Token ID'] == tgt_id]
            # print(row_tgt)
    
            # print(aligned_tks_df['Token ID'] )
            # print(row_tgt)
            # print(row_en.iloc[0]['Token ID '+tgt_lan])
            if row_en.iloc[0]['Token ID '+tgt_lan] is not None:
                aligned_tks_df.loc[row_en.index, 'Token ID '+tgt_lan] += '➕' + row_tgt.iloc[0]['Token ID']
                aligned_tks_df.loc[row_en.index, 'Token '+tgt_lan] += '➕' + row_tgt.iloc[0]['Token']
                if aligned_tks_df.loc[row_en.index, 'POS '+tgt_lan].item() != process_nan(row_tgt.iloc[0]['POS']):
                    aligned_tks_df.loc[row_en.index, 'POS '+tgt_lan] += '➕' + process_nan(row_tgt.iloc[0]['POS'])
                if aligned_tks_df.loc[row_en.index, 'Sense '+tgt_lan].item() != process_nan(row_en.iloc[0]['BN Synset']):
                    aligned_tks_df.loc[row_en.index, 'Sense '+tgt_lan] += '➕' + process_nan(str(row_en.iloc[0]['BN Synset']).split(";")[0])
            else:
                # print(str(row_en.iloc[0]['BN Synset']).split(";")[0])
                # print(row_tgt.iloc[0]['Token ID'])
                # print(row_en.index.values)
                aligned_tks_df.loc[row_en.index.values[0], 'Token ID '+tgt_lan] = row_tgt.iloc[0]['Token ID']
                # print(aligned_tks_df.loc[row_en.index, 'Token ID '+tgt_lan],row_tgt.iloc[0]['Token ID'],'\\n\\n')
        
                aligned_tks_df.loc[row_en.index.values[0], 'Token '+tgt_lan] = row_tgt.iloc[0]['Token']
                aligned_tks_df.loc[row_en.index.values[0], 'Lemma '+tgt_lan] = row_tgt.iloc[0]['Lemma']
                aligned_tks_df.loc[row_en.index.values[0], 'POS '+tgt_lan] = row_tgt.iloc[0]['POS']
                aligned_tks_df.loc[row_en.index.values[0], 'Sense '+tgt_lan] = str(row_en.iloc[0]['BN Synset']).split(";")[0]
                aligned_tks_df.loc[row_en.index.values[0], 'BN Synset'] = str(row_en.iloc[0]['BN Synset']).split(";")[0]
        except:
            print(f"error {i}")
    # count_en += len(en_sents[sentence_num])
    # count_es += len(tgt_sents[sentence_num])
    # sentence_num += 1


In [47]:
# count_en = 0
# sentence_num = 0
# count_es = 0
# for i, row in trans_df.iterrows():
#     # print(row.iloc[0])
#     sent_i = row.iloc[0]
#     # print(sent_i)
#     # print(sent_i)
#     tgt_sent_i = sent_i
#     # print(row)
#     # sent_i = sent_i.replace("i","d")
#     tgt_sent = tkns_df.loc[count_es:count_es+len(tgt_sents[sentence_num])]
#     en_sent = aligned_tks_df.loc[count_en:count_en+len(en_sents[sentence_num])]
#     # print(tgt_sent.loc[2])
#     # print(row)
#     for alignment in row['English-'+language+' Alignment'].split():
#         w1, w2 = alignment.split('-')
#         row_tgt = tgt_sent.loc[count_es+int(w2)]
#         row_en = en_sent.loc[count_en+int(w1)]
#         if row_en['Token ID '+tgt_lan] is not None:
#             if str(aligned_tks_df.loc[count_en+int(w1), 'Token ID '+tgt_lan]) != "nan":
#                 aligned_tks_df.loc[count_en+int(w1), 'Token ID '+tgt_lan] += '➕' + str(row_tgt['Token ID'])
#             else:
#                 aligned_tks_df.loc[count_en+int(w1), 'Token ID '+tgt_lan] = str(row_tgt['Token ID'])
#             aligned_tks_df.loc[count_en+int(w1), 'Token '+tgt_lan] += '➕' + row_tgt['Token']
#             if aligned_tks_df.loc[count_en+int(w1), 'POS '+tgt_lan] != process_nan(row_tgt['POS']):
#                 aligned_tks_df.loc[count_en+int(w1), 'POS '+tgt_lan] += '➕' + process_nan(row_tgt['POS'])
#             if aligned_tks_df.loc[count_en+int(w1), 'Sense '+tgt_lan] != process_nan(row_en['BN Synset']):
#                 if str(aligned_tks_df.loc[count_en+int(w1), 'Sense '+tgt_lan]) != "nan":
#                     aligned_tks_df.loc[count_en+int(w1), 'Sense '+tgt_lan] += '➕' + process_nan(row_en['BN Synset'])
#         else:
#             aligned_tks_df.loc[count_en+int(w1), 'Token ID '+tgt_lan] = row_tgt['Token ID']
#             aligned_tks_df.loc[count_en+int(w1), 'Token '+tgt_lan] = row_tgt['Token']
#             aligned_tks_df.loc[count_en+int(w1), 'Lemma '+tgt_lan] = row_tgt["Lemma"]
#             aligned_tks_df.loc[count_en+int(w1), 'POS '+tgt_lan] = row_tgt['POS']
#             aligned_tks_df.loc[count_en+int(w1), 'Sense '+tgt_lan] = row_en['BN Synset']
# 
#     count_en += len(en_sents[sentence_num])
#     count_es += len(tgt_sents[sentence_num])
#     sentence_num += 1
#     # print(count_en)

In [48]:
# aligned_tks_df[]
aligned_tks_df.head(2000)

,Token ID,Token,Lemma,POS,BN Synset,Token ID BN,Token BN,Lemma BN,POS BN,Sense BN
0,d001.s001.t001,This,NaN,X,nan,d001.s001.t001,এই,এই,X,nan
1,d001.s001.t002,document,document,N,bn:00028015n;wiki:document;wn:document%1:10:00...,None,None,None,None,None
2,d001.s001.t003,is,be,V,NaN,None,None,None,None,None
3,d001.s001.t004,a,NaN,X,NaN,None,None,None,None,None
4,d001.s001.t005,summary,summary,N,bn:00075142n,d001.s001.t010,সারাংশ,সারাংশ,N,bn:00075142n
...,...,...,...,...,...,...,...,...,...,...
1995,d003.s016.t014,and,NaN,X,nan,d003.s016.t011,এবং,এবং,X,nan
1996,d003.s016.t015,provide,provide,V,bn:00084692v;wn:cater%2:34:00::;wn:ply%2:34:00...,None,None,None,None,None
1997,d003.s016.t016,the,NaN,X,NaN,None,None,None,None,None
1998,d003.s016.t017,backdrop,backdrop,N,nan,d003.s016.t018,পটভূমি,পটভূমি,N,nan


In [49]:
aligned_tks_df.to_csv(ALIGN_TOKEN_FILE, sep='\t', index=False)